In [1]:
import pandas as pd
import re
import pycountry
import json
import dask.dataframe as dd

def extract_numeric_year(column_name):
  match = re.match(r'(\d+) \[YR(\d+)\]', column_name)
  if match:
    return match.group(1)
  else:
    return column_name
      
# Load the datasets
fertility_df = pd.read_csv("Resources/Population and Ferility Data by Country.csv")
gdp_df = pd.read_csv("Resources/GDP by Country_Data.csv")

In [2]:
# Apply the function to extract numeric year from column names
fertility_df.columns = fertility_df.columns.map(extract_numeric_year)
gdp_df.columns = gdp_df.columns.map(extract_numeric_year)

In [3]:
#Check for missing values in both datasets
fertility_missing = fertility_df.isnull().sum()
gdp_missing = gdp_df.isnull().sum()

print("Fertility Data Missing Values:")
print(fertility_missing)
print("\nGDP Data Missing Values:")
print(gdp_missing)

Fertility Data Missing Values:
Series Name     3
Series Code     5
Country Name    5
Country Code    5
1960            5
               ..
2019            5
2020            5
2021            5
2022            5
2023            5
Length: 68, dtype: int64

GDP Data Missing Values:
Country Name    3
Country Code    5
Series Name     5
Series Code     5
1960            5
               ..
2019            5
2020            5
2021            5
2022            5
2023            5
Length: 68, dtype: int64


In [4]:
# Ensure year columns are integers (Some columns might have extra spaces or different formats)
fertility_df.columns = [col.strip() for col in fertility_df.columns]  # Remove any extra spaces
gdp_df.columns = [col.strip() for col in gdp_df.columns]  # Remove any extra spaces

In [5]:
# Reshape the data (pivot the data so we can have years as columns)
fertility_long = pd.melt(fertility_df, id_vars=["Series Name", "Country Name", "Country Code"], 
                         var_name="Year", value_name="Fertility Rate")
gdp_long = pd.melt(gdp_df, id_vars=["Country Name", "Country Code", "Series Name", "Series Code"], 
                   var_name="Year", value_name="GDP")

In [6]:
# Clean the Year column (remove non-integer values)
fertility_long['Year'] = pd.to_numeric(fertility_long['Year'], errors='coerce')
gdp_long['Year'] = pd.to_numeric(gdp_long['Year'], errors='coerce')

In [7]:
# Remove Decimal and Drop NaN values
fertility_long.dropna(subset=['Year'], inplace=True)
gdp_long.dropna(subset=['Year'], inplace=True)
fertility_long['Year'] = fertility_long['Year'].astype(int)
gdp_long['Year'] = gdp_long['Year'].astype(int)

In [8]:
# Merge the datasets by Country Name, Country Code, and Year
merged_df = pd.merge(fertility_long, gdp_long, on=["Country Name", "Country Code", "Year"], how="inner")

In [9]:
# Ensure the data types are correct (e.g., convert Year to integer, GDP to numeric)
merged_df['Year'] = merged_df['Year'].astype(int)
merged_df['Fertility Rate'] = pd.to_numeric(merged_df['Fertility Rate'], errors='coerce')
merged_df['GDP'] = pd.to_numeric(merged_df['GDP'], errors='coerce')

In [10]:
# Replace missing values with 0
merged_df['Fertility Rate'] = merged_df['Fertility Rate'].fillna(0)
merged_df['GDP'] = merged_df['GDP'].fillna(0)

In [11]:
# Create a list of valid country names
valid_countries = [country.name for country in pycountry.countries]

# Filter rows based on country names
merged_df = merged_df[merged_df['Country Name'].isin(valid_countries)]

In [12]:
# Filter the dataset for the years 2014 to 2024
filtered_df = merged_df[merged_df['Year'].between(2014, 2024)]

In [13]:
# Remove duplicate countries by keeping only 'GDP per capita (current US$)' rows
filtered_df = merged_df[merged_df['Series Name_y'] == 'GDP per capita (current US$)'].copy()

# Extract the GDP growth values
gdp_growth_df = merged_df[merged_df['Series Name_y'] == 'GDP per capita growth (annual %)'][['Country Code', 'GDP']].copy()
gdp_growth_df.rename(columns={'GDP': 'GDP Percentage'}, inplace=True)

In [14]:
# Merge the GDP Percentage data with the filtered DataFrame
final_df = filtered_df.merge(gdp_growth_df, on='Country Code', how='left')

ArrowMemoryError: malloc of size 376197120 failed

In [ ]:
# Save the filtered data to a CSV and JSONfile
filtered_df.to_csv('Resources/filtered_population_fertility_gdp_2014_2024.csv', index=False)
filtered_df.to_json('Resources/filtered_population_fertility_gdp_2014_2024.json', orient='records', lines=True)

In [ ]:
# Show the first few rows of the filtered data to verify
filtered_df.tail()